### POS tagging with maximum entropy models (10 pts)

In this task you will build a maximum entropy model for part-of-speech tagging. As the name suggests, our problem is all about converting a sequence of words into a sequence of part-of-speech tags. 
<img src=https://i.stack.imgur.com/6pdIT.png width=320>


__Your man goal:__ implement the model from [the article you're given](W96-0213.pdf).

Unlike previous tasks, this one gives you greater degree of freedom and less automated tests. We provide you with programming interface but nothing more.

__A piece of advice:__ there's a lot of objects happening here. If you don't understand why some object is needed, find `def train` function and see how everything is linked together.


### Part I: reading input data

In [117]:
# cityhash murmurhash
# hash lists, pypy
from collections import OrderedDict
import conllu
import itertools
import numpy as np
import cit

In [80]:
# Data types:
# Word: str
# Sentence: list of str
TaggedWord = collections.namedtuple('TaggedWord', ['text', 'tag'])
# TaggedSentence: list of TaggedWord
# Tags: list of TaggedWord
# TagLattice: list of Tags
# Tagging: list of TaggedSentence


def read_tagged_sentences(path):
    """
    Read tagged sentences from CoNLL-U file and return array of TaggedSentence (array of lists of TaggedWord).
    """
    result = []
    with open(path, encoding='utf-8') as f:
        sentences = conllu.parse_incr(f)
        for sentence in sentences:
#             return sentence
            result.append([TaggedWord(text=word['form'], tag=word['upostag']) for word in sentence])
    return result

In [116]:
tagged_sentences = read_tagged_sentences('data/en-ud-train.conllu')

In [89]:
def write_tagged_sentence(tagged_sentence, f):
    """
    Write tagged sentence in CoNLL-U format to file-like object f.
    """
    token_list = conllu.models.TokenList([
        OrderedDict([('id', i+1),
             ('form', word.text),
             ('lemma', '-'),
             ('upostag', word.tag),
             ('xpostag', '-'),
             ('feats', OrderedDict([('Number', 'Sing')])),
             ('head', 0),
             ('deprel', '-'),
             ('deps', None),
             ('misc', OrderedDict([('SpaceAfter', 'No')]))])
        for i, word in enumerate(tagged_sentence)
    ]
    )
    token_list.metadata = OrderedDict([('text', ' '.join([word.text for word in tagged_sentence]))])
    f.write(token_list.serialize())

In [92]:
_ = open('data/test.conllu', mode='w', encoding='utf-8')
with open('data/test.conllu', mode='a', encoding='utf-8') as f:    
    write_tagged_sentence(tagged_sentences[0], f)
    write_tagged_sentence(tagged_sentences[1], f)

In [97]:
def read_tags(path):
    """
    Read a list of possible tags from file and return the list.
    """
    with open(path, encoding='utf-8') as f:
        result = [line for line in f.read().split('\n')]
    return result[:-1]

### Part II: evaluation

We want you to estimate tagging quality by a simple accuracy: a fraction of tag predictions that turned out to be correct - averaged over the entire training corpora.

In [106]:
# Data types:
TaggingQuality = collections.namedtuple('TaggingQuality', ['acc'])

def tagging_quality(ref, out):
    """
    Compute tagging quality and reutrn TaggingQuality object.
    """
    nwords = 0
    ncorrect = 0
    import itertools
    for ref_sentence, out_sentence in itertools.zip_longest(ref, out):
        for ref_word, out_word in itertools.zip_longest(ref_sentence, out_sentence):
            nwords += 1
            if ref_word.tag == out_word.tag:
                ncorrect += 1
    return ncorrect / nwords

### Part III: Value and Update

In order to implement two interlinked data structures: 
* __Value__ - a class that holds POS tagger's parameters. Basically an array of numbers
* __Update__ - a class that stores updates for Value

In [123]:
class Value:
    def __init__(self, n):
        """
        Dense object that holds parameters.
        :param n: array length
        """
        self.values = np.zeros(n)

    def dot(self, update):
        return self.values[update.positions].dot(update.values)

    def assign(self, other):
        """
        self = other
        other is Value.
        """
        self.values = other.values

    def assign_mul(self, coeff):
        """
        self = self * coeff
        coeff is float.
        """
        self.values = self.values * coeff

    def assign_madd(self, x, coeff):
        """
        self = self + x * coeff
        x can be either Value or Update.
        coeff is float.
        """
        if type(x) == Update:
            self.values[x.positions] = self.values[x.positions] + x.values * coeff
        else:
            self.values = self.values + x.values * coeff


class Update:
    """
    Sparse object that holds an update of parameters.
    """

    def __init__(self, positions=None, values=None):
        """
        positions: array of int
        values: array of float
        """
        self.positions = np.array(positions, dtype = np.int32)
        self.values = np.array(values, dtype = np.float32)

    def assign_mul(self, coeff):
        """
        self = self * coeff
        coeff: float
        """
        self.values = self.values * coeff

    def assign_madd(self, update, coeff):
        """
        self = self + update * coeff
        coeff: float
        """
        self.values = self.values + update.values * coeff

### Part IV: Maximum Entropy POS Tagger
_step 1 - draw an oval; step 2 - draw the rest of the owl (c)_

In this secion you will implement a simple linear model to predict POS tags.
Make sure you [read the article](W96-0213.pdf) before you proceed.

In [138]:
# Data Types:
Features = Update
Hypo = collections.namedtuple('Hypo', ['prev', 'pos', 'tagged_word', 'score'])

# prev: previous Hypo
# pos: position of word (0-based)
# tagged_word: tagging of source_sentence[pos]
# score: sum of scores over edges

TaggerParams = collections.namedtuple('FeatureParams', [
    'src_window',
    'dst_order',
    'max_suffix',
    'beam_size',
    'nparams'
    ])

import cityhash
def h(x):
    """
    Compute CityHash of any object.
    Can be used to construct features.
    """
#     hash(repr(x))
    return cityhash.CityHash64(repr(x))

In [136]:
class LinearModel:
    """
    A thing that computes score and gradient for given features.
    """

    def __init__(self, n):
        self._params = Value(n)

    def params(self):
        return self._params

    def score(self, features):
        """
        features: Update
        """
        return self._params.dot(features)

    def gradient(self, features, score):
        return features

In [185]:
class FeatureComputer:
    def __init__(self, tagger_params, source_sentence):
        """
        MY
        source_sentence: list of tokens, including punct marks
        """
        self.nparams = tagger_params.nparams
        self.source_sentence = source_sentence

    def compute_features(self, hypo):
        """
        Compute features for a given Hypo and return Update.
        """
        def append_pos_and_value(positions, values, feature):
            new_pos = h(feature) % self.nparams
            positions.append(new_pos)
            values.append(1.)
        
        positions = []
        values = []
        pos = hypo.pos
        w = hypo.tagged_word.text
        t = hypo.tagged_word.tag
        src_length = len(self.source_sentence)
        
        append_pos_and_value(positions, values, f'wi={w}|ti={t}')
        if pos > 0:
            append_pos_and_value(positions, values, f'tim1={hypo.prev.tagged_word.tag}|ti={t}')
            append_pos_and_value(positions, values, f'wim1={hypo.prev.tagged_word.text}|ti={t}')
        if pos > 1:
            append_pos_and_value(positions, values, f'tim1tim2={hypo.prev.tagged_word.tag}{hypo.prev.prev.tagged_word.tag}|ti={t}')
            append_pos_and_value(positions, values, f'wim2={hypo.prev.prev.tagged_word.text}|ti={t}')
            
        if pos < src_length-1:
            append_pos_and_value(positions, values, f'wip1={self.source_sentence[pos+1]}|ti={t}')
        if pos < src_length-2:
            append_pos_and_value(positions, values, f'wip2={self.source_sentence[pos+2]}|ti={t}')
            
        if '-' in t:
            append_pos_and_value(positions, values, f'CH|ti={t}')
        if any(char.isdigit() for char in t):
            append_pos_and_value(positions, values, f'CN|ti={t}')
        if any(char.isupper() for char in t):
            append_pos_and_value(positions, values, f'CU|ti={t}')
        for i in range(min(len(t)+1, 5)):
            append_pos_and_value(positions, values, f'pr={t[:i]}|ti={t}')
        for i in range(min(len(t)+1, 5)):
            append_pos_and_value(positions, values, f'sf={t[len(t)-i:]}|ti={t}')
            
        return Update(positions, values)

### Part V: Beam search

We can find the most likely tagging approximately using Beam Search. As everything else, it comes with a separate interface.

In [ ]:
class BeamSearchTask:
    """
    An abstract beam search task. Can be used with beam_search() generic 
    function.
    """

    def __init__(self, tagger_params, source_sentence, model, tags):
        self.tagger_params = tagger_params
        self.source_sentence = source_sentence
        self.model = model
        self.tags = tags
        self.feature_computer = FeatureComputer(tagger_params, source_sentence)

    def total_num_steps(self):
        """
        Number of hypotheses between beginning and end (number of words in
        the sentence).
        """
        return len(source_sentence)

    def beam_size(self):
        return self.tagger_params.beam_size

    def expand(self, hypo):
        """
        Given Hypo, return a list of its possible expansions.
        'hypo' might be None -- return a list of initial hypos then.

        Compute hypotheses' scores inside this function!
        """
        possible_hypos = []
        for tag in self.tags:
            tagged_word = TaggedWord(text=self.source_sentence[hypo.pos+1], tag=tag)
            new_hypo = Hypo(prev=hypo, pos=hypo.pos+1, tagged_word=tagged_word, score=0)
            new_features = self.feature_computer.compute_features(new_hypo)
            new_score = hypo.score + self.model.score(new_features)
            possible_hypos.append(new_hypo._replace(score=new_score))
        return possible_hypos

    def recombo_hash(self, hypo):
        """
        If two hypos have the same recombination hashes, they can be collapsed
        together, leaving only the hypothesis with a better score.
        """
        <YOUR CODE>


def beam_search(beam_search_task):
    """
    Return list of stacks.
    Each stack contains several hypos, sorted by score in descending 
    order (i.e. better hypos first).
    """
    <YOUR CODE>


In [ ]:
def tag_sentences(dataset, <YOUR_PARAMS>):
    """
    Main predict function.
    Tags all sentences in dataset. Dataset is a list of TaggedSentence; while 
    tagging, ignore existing tags.
    """
    <YOUR CODE>


### Part VI: Optimization objective and algorithm

Once we defined our model and inference algorithm, we can define an optimization task: an object that computes loss function and its gradients w.r.t. model parameters.

In [ ]:
class OptimizationTask:
    """
    Optimization task that can be used with sgd().
    """

    def params(self):
        """
        Parameters which are optimized in this optimization task.
        Return Value.
        """
        raise NotImplementedError()

    def loss_and_gradient(self, golden_sentence):
        """
        Return (loss, gradient) on a specific example.

        loss: float
        gradient: Update
        """
        raise NotImplementedError()


class UnstructuredPerceptronOptimizationTask(OptimizationTask):
    def __init__(self, ...):
        <YOUR CODE>

    def params(self):
        <YOUR CODE>

    def loss_and_gradient(self, golden_sentence):
        <YOUR CODE>


class StructuredPerceptronOptimizationTask(OptimizationTask):
    def __init__(self, tagger_params, tags):
        self.tagger_params = tagger_params
        self.model = LinearModel(...)
        self.tags = tags

    def params(self):
        return self.model.params()

    def loss_and_gradient(self, golden_sentence):
        # Do beam search.
        beam_search_task = BeamSearchTask(
            self.tagger_params, 
            [golden_tagged_word.text for golden_tagged_word in golden_sentence], 
            self.model, 
            self.tags
            )
        stacks = beam_search(beam_search_task)

        # Compute chain of golden hypos (and their scores!).
        golden_hypo = None
        feature_computer = ...
        for i in range(len(golden_sentence)):
            new_golden_hypo = ...
            golden_hypo = golden_hypo

        # Find where to update.
        golden_head = <YOUR CODE>
        rival_head = <YOUR CODE>

        # Compute gradient.
        grad = Update()
        while golden_head and rival_head:
            rival_features = feature_computer.compute_features(rival_head)
            grad.assign_madd(self.model.gradient(rival_features, score=None), 1)

            golden_features = feature_computer.compute_features(golden_head)
            grad.assign_madd(self.model.gradient(golden_features, score=None), -1)


            golden_head = golden_head.prev
            rival_head = rival_head.prev

        return grad


### Part VII: optimizer

By this point we can define a model with parameters $\theta$ and a problem that computes gradients $ \partial L \over \partial \theta $ w.r.t. model parameters.

Optimization is performed by gradient descent: $ \theta := \theta - \alpha {\partial L \over \partial \theta} $

In order to speed up training, we use stochastic gradient descent that operates on minibatches of data.

In [ ]:
SGDParams = collections.namedtuple('SGDParams', [
    'epochs',
    'learning_rate',
    'minibatch_size',
    'average' # bool or int
    ])


def make_batches(dataset, minibatch_size):
    """
    Make list of batches from a list of examples.
    """
    <YOUR CODE>


def sgd(sgd_params, optimization_task, dataset, after_each_epoch_fn):
    """
    Run (averaged) SGD on a generic optimization task. Modify optimization
    task's parameters.

    After each epoch (and also before and after the whole training),
    run after_each_epoch_fn().
    """
    <YOUR CODE>


### Part VIII: Training loop

The train function combines everthing you used below to get new 

In [ ]:
def train(
    tags='./data/tags',
    train_dataset='./data/en-ud-train.conllu',
    dev_dataset='./data/en-ud-dev.conllu',
    model='./model.npz',
    
    sgd_epochs=15,
    sgd_learning_rate=0.01,
    sgd_minibatch_size=32,
    
    # Number of context tags in output tagging to use for features
    tagger_src_window=2,
    
    # Number of context tags in output tagging to use for features
    tagger_dst_order=3,
    
    # Maximal number of prefix/suffix letters to use for features
    tagger_max_suffix=4,
    
    # Width for beam search (0 means unstructured)
    beam_size=1,
    
    # Parameter vector size (for hashing)
    nparams= 2 * 22,
):
    """ Train a pos-tagger model and save it's parameters to :model: """

    # Beam size.
    optimization_task_cls = StructuredPerceptronOptimizationTask
    if beam_size == 0:
        beam_size = 1
        optimization_task_cls = UnstructuredPerceptronOptimizationTask

    # Parse cmdargs.
    tags = read_tags(cmdargs.tags)
    train_dataset = read_tagged_sentences(train_dataset)
    dev_dataset = read_tagged_sentences(dev_dataset)
    params = None
    if os.path.exists(cmdargs.model):
        params = pickle.load(open(cmdargs.model, 'rb'))
    sgd_params = SGDParams(
        epochs=sgd_epochs,
        learning_rate=sgd_learning_rate,
        minibatch_size=sgd_minibatch_size,
        average=sgd_average
        )
    tagger_params = TaggerParams(
        src_window=tagger_src_window,
        dst_order=tagger_dst_order,
        max_suffix=tagger_max_suffix,
        beam_size=beam_size,
        nparams=nparams
        )

    # Load optimization task
    optimization_task = optimization_task_cls(...)
    if params is not None:
        print('\n\nLoading parameters from %s\n\n' % cmdargs.model)
        optimization_task.params().assign(params)

    # Validation.
    def after_each_epoch_fn():
        model = LinearModel(cmdargs.nparams)
        model.params().assign(optimization_task.params())
        tagged_sentences = tag_sentences(dev_dataset, <YOUR_PARAMS>)
        q = pprint.pformat(tagging_quality(out=tagged_sentences, ref=dev_dataset))
        print()
        print(q)
        print()

        # Save parameters.
        print('\n\nSaving parameters to %s\n\n' % cmdargs.model)
        pickle.dump(optimization_task.params(), open(cmdargs.model, 'wb'))

    # Run SGD.
    sgd(sgd_params, optimization_task, train_dataset, after_each_epoch_fn)


In [ ]:
# train a model with default params
train(model='./default_model.npz')

### Part IX: Evaluate the trained model

In [4]:
def test(
    tags='./data/tags',
    dataset='./data/en-ud-dev.conllu',
    model='./model.npz',
    
    # model and inference params; see train for their description
    tagger_src_window=2,
    tagger_dst_order=3,
    tagger_max_suffix=4,
    beam_size=1,
):


    tags = read_tags(tags)
    dataset = read_tagged_sentences(dataset)
    params = pickle.load(open(model, 'rb'))
    tagger_params = TaggerParams(
        src_window=tagger_src_window,
        dst_order=tagger_dst_order,
        max_suffix=tagger_max_suffix,
        beam_size=beam_size,
        nparams=0
        )

    # Load model.
    model = LinearModel(params.values.shape[0])
    model.params().assign(params)

    # Tag all sentences.
    tagged_sentences = tag_sentences(dataset, <YOUR_PARAMS>)

    # Write tagged sentences.
    for tagged_sentence in tagged_sentences:
        write_tagged_sentence(tagged_sentence, sys.stdout)

    # Measure and print quality.
    q = pprint.pformat(tagging_quality(out=tagged_sentences, ref=dataset))
    print(q, file=sys.stderr)


SyntaxError: invalid syntax (<ipython-input-4-c5d1753b0c3f>, line 30)

In [ ]:
# test 
test(model='./default_model.npz')

# sanity chec: accuracy > 90%.

### Part X: play with it

_This part is optional_

Once you've built something, it's only natural to test the limits of your contraption.

At minumum, we want you to find out how default model accuracy depends on __beam size__

To get maximum points, your model should get final quality >= 93% 

Any further analysis is welcome, as always.

In [ ]:
<YOUR CODE>